<a href="https://colab.research.google.com/github/roman-bagdasarian/Classifying-Quantum-Phases-of-Matter/blob/main/FLIQ_Challenge_ClassiqDuQIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Make Sure You Are Ready to Go

$\renewcommand{\ket}[1]{\left| #1 \right\rangle}
\renewcommand{\bra}[1]{\left\langle #1 \right|}
\renewcommand{\braket}[2]{\left\langle #1 | #2 \right\rangle}
\newcommand{\ketbra}[2]{\left| #1 \right\rangle\!\left\langle #2 \right|}$

If you haven't done it yet, try running the following lines of code and use the [registration and installation](https://docs.classiq.io/latest/classiq_101/registration_installations/) page if you are having difficulty setting up your environment.\
Uncomment and run the following command to install or update to the latest version of the Classiq SDK (if not installed yet):

In [ ]:
pip install -U classiq

In [ ]:
import classiq
#from classiq.builtin import Measure

Uncomment and run the following command if your machine has not been

authenticated yet, you only need to run it once!

In [ ]:
pip install keyrings.alt


In [ ]:
#from keyrings.alt.file import PlaintextKeyring

# # Use a basic keyring backend that stores credentials insecurely (for environments like Colab or VS Code)
#keyring.set_keyring(PlaintextKeyring())

# import classiq
classiq.authenticate()

Your user code: QKJK-RNHV
If a browser doesn't automatically open, please visit this URL from any trusted device: https://auth.classiq.io/activate?user_code=QKJK-RNHV
Please set a password for your new keyring: ··········
Please confirm the password: ··········


Now you are good to go!

# Rydberg Phase Diagram

Before starting to code, let us reiterating some theory on Rydberg atoms - the subject of this challenge. They interact via the following Hamiltonian:

$$
H = \frac{\Omega}{2} \sum_{i=1}^N X_i
    - \delta \sum_{i=1}^N n_i
    + \sum_{i \lt j} \frac{\Omega R_b^6 }{(a|i-j|)^6} n_i n_j.
$$

You can find the phase diagram for a $51$-atom chain below. It is obtained by fixing $a=1$ and $\Omega=1$ and varying $\delta$ and $R_b$.

<img src="phase_diagram.png" alt="Phase Diagram" width="800">


Fig.1: Phase diagram of the 1D Rydberg Hamiltonian, traced out by (left) bipartite entanglement entropy and (right) expectation value of the number of Rydberg excitations. Plots are obtained using tensor-network representation of the ground states of $H$.

In this challenge, we focus on distinguishing between the $Z2$ phase, where the ground state of $H$ has large overlap with the state $\ket{rgr\ldots gr}$, and the $Z3$ phase, where the ground state overlaps strongly with basis states of the form $\ket{\ldots rggrgg\ldots}$.

Evidently, such systems can be efficiently studied using tensor networks. However, this challenge prepares us for a more realistic scenario in which we only have access to measurement outcomes from the ground state of some Hamiltonian, and our goal is to determine which phase of matter the state belongs to.

# Loading and Processing Measurement Data

*   List item
*   List item



Training data for your model contains measurement results in randomized bases performed on a 51-qubit Rydberg atoms chain. We load training data from the .npz file in the next cell.

In [ ]:
!wget https://github.com/dmitriikhitrin/Classiq-x-DuQIS-FLIQ-Challenge/raw/main/training_data.npz


--2025-05-17 13:48:07--  https://github.com/dmitriikhitrin/Classiq-x-DuQIS-FLIQ-Challenge/raw/main/training_data.npz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dmitriikhitrin/Classiq-x-DuQIS-FLIQ-Challenge/main/training_data.npz [following]
--2025-05-17 13:48:07--  https://raw.githubusercontent.com/dmitriikhitrin/Classiq-x-DuQIS-FLIQ-Challenge/main/training_data.npz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1021888 (998K) [application/octet-stream]
Saving to: ‘training_data.npz.1’

training_data.npz.1 100%[===================>] 997.94K  --.-KB/s    in 0.06s   

2025-05-17 13:48:07 (15

In [ ]:
import numpy as np
# You might need to make additional imports depending on your implementation

In [ ]:
loaded = np.load("training_data.npz", allow_pickle=True)


unprocessed_features = loaded["features"].tolist()
unprocessed_labels = loaded["labels"].tolist()

print(f'There are {len(unprocessed_features)} data points')
print(f'There were T = {len(unprocessed_features[0])} measurements performed for each data point')
print(f'The measurements were performed on {len(unprocessed_features[0][0])} qubits')
print(f'Example: 2nd experiment result of 8th data point -> {unprocessed_features[7][1]}')
print(f'Example: label for the 8th data point -> {unprocessed_labels[7]}')

There are 20 data points
There were T = 500 measurements performed for each data point
The measurements were performed on 51 qubits
Example: 2nd experiment result of 8th data point -> ['r', '-', 'i', 'i', 'r', '+', 'r', 'g', '+', '-i', 'r', '-', '-', 'g', '-i', '-', 'r', 'g', 'i', 'i', 'i', '-', 'r', '-i', 'r', 'i', 'r', '+', 'i', 'g', '-', '-i', '-', 'g', '-i', 'i', 'r', '-', '-', '-', 'i', 'i', '-i', 'g', '-i', 'g', 'r', '-', 'r', '+', '-']
Example: label for the 8th data point -> Z2


In the above,
- $\ket{g}$ is the atomic ground state, which is a $+1$-eigenstate of Pauli $Z$
- $\ket{r}$ is the highly excited Rydberg state, which is a $-1$-eigenstate of Pauli $Z$
- $\ket{+} = \frac{1}{\sqrt2}(\ket{g} + \ket{r})$, a $+1$-eigenstate of Pauli $X$
- $\ket{-} = \frac{1}{\sqrt2}(\ket{g} - \ket{r})$, a $-1$-eigenstate of Pauli $X$
- $\ket{+i} = \frac{1}{\sqrt2}(\ket{g} +i\ket{r})$, a $+1$-eigenstate of Pauli $Y$
- $\ket{-i} = \frac{1}{\sqrt2}(\ket{g} -i \ket{r})$, a $-1$-eigenstate of Pauli $Y$.

It is up to you how to convert the features into classical shadows and labels into numbers and then both into training data for your model. For example, you could assign $-1$ to $Z2$ and $+1$ to $Z3$.

**Note:** If you decide to define any helper classes/functions in a separate Python file, please submit it alongside your solution notebook, so we can run and grade it properly

In [ ]:
### Your Code Goes Here: ###

import numpy as np

# ——— 1) Define the 6 single-qubit basis states ———
g    = np.array([1, 0], dtype=complex)                   # |g⟩, Z=+1
r    = np.array([0, 1], dtype=complex)                   # |r⟩, Z=–1
plus  = (1/np.sqrt(2)) * np.array([1,  1], dtype=complex)  # |+⟩, X=+1
minus = (1/np.sqrt(2)) * np.array([1, -1], dtype=complex)  # |–⟩, X=–1
p_i   = (1/np.sqrt(2)) * np.array([1,  1j], dtype=complex) # |+i⟩, Y=+1
m_i   = (1/np.sqrt(2)) * np.array([1, -1j], dtype=complex) # |–i⟩, Y=–1

basis_states = {'g':g,'r':r,'+':plus,'-':minus,'+i':p_i,'-i':m_i}
I2 = np.eye(2, dtype=complex)

def sigma_op(v):
    """Return classical‐shadow operator σ = 3|v⟩⟨v| – I."""
    P = np.outer(v, v.conj())
    return 3*P - I2

# precompute
sigma_dict = {k: sigma_op(v) for k,v in basis_states.items()}


# ——— 2) Make sure you already have these from your earlier cell: ———
# unprocessed_features : list of 20 samples × 500 shots × 51 strings
# unprocessed_labels   : list of 20 labels in {'Z2','Z3'}

num_samples = len(unprocessed_features)
T           = len(unprocessed_features[0])      # ≈500
n_qubits    = len(unprocessed_features[0][0])   # 51

# ——— 3) Build 1‐qubit shadow estimates with normalization ———
shadows = np.zeros((num_samples, n_qubits, 2, 2), dtype=complex)

for i, shots in enumerate(unprocessed_features):
    acc = np.zeros((n_qubits, 2, 2), dtype=complex)
    for shot in shots:
        for q, outcome in enumerate(shot):
            # ---- normalize the string ----
            o = outcome.strip()
            # convert any Unicode minus (U+2212) to ASCII hyphen
            o = o.replace('−','-')
            # map lone 'i' to '+i'
            if o == 'i':
                o = '+i'
            # sanity check
            if o not in sigma_dict:
                raise KeyError(f"Unknown outcome {outcome!r} → normalized {o!r}")
            acc[q] += sigma_dict[o]
    shadows[i] = acc / T

# ——— 4) Flatten into feature‐matrix X ———
X = shadows.reshape(num_samples, -1)    # shape (20, 51×4)

# (optional) split real/imag if your model only takes reals:
# X = np.concatenate([X.real, X.imag], axis=1)

# ——— 5) Encode labels ———
label_map = {'Z2': -1, 'Z3': +1}
y = np.array([label_map[L] for L in unprocessed_labels], dtype=int)

print("X.shape:", X.shape)
print("y.shape:", y.shape, " (–1→Z2, +1→Z3)")


X.shape: (20, 204)
y.shape: (20,)  (–1→Z2, +1→Z3)


# Defining a Quantum Model

In this section, you will create a QML model for classifying the quantum phases. This will include 3 stages:
- First, you will need to decide on the data encoding scheme, i.e. loading numerical features you obtained above into the quantum circuit.
- Then, you will need to come up with an ansatz - a parametrized quantum circuit, which will be optimized to perform classification.
- Finally, to readout classical information from the quantum model, you will need to perform some sort of measurement on the resultant quantum state. Perhaps, you could extract an expectation value of some Pauli-string $P \in \{I, X, Y, Z\}^{\otimes N}$, so that $\langle P \rangle < b$ is interpreted as $Z2$ and  $\langle P \rangle > b$ is interpreted as $Z3$ for some decision boundary $b$.

There are several approaches to QML in Classiq, linked below.

You may find the following guides useful:
- QML with Classiq: http://docs.classiq.io/latest/user-guide/read/qml_with_classiq_guide/
- Variational Model Example: https://github.com/Classiq/classiq-library/blob/main/algorithms/qaoa/maxcut/qaoa_max_cut.ipynb
- Hybrid QNN: https://docs.classiq.io/latest/explore/algorithms/qml/hybrid_qnn/hybrid_qnn_for_subset_majority/

Although the 2nd guide describes a hybrid model, **you may not implement a hybrid model**, the guide should only be used as a reference as to how to implement QML.

**Warning**: Training using the Classiq PyTorch integration may take a prohibitive amount of time. Consider this when choosing an approach.

In [ ]:
from classiq import *
from classiq.execution import *
import numpy as np
from classiq.synthesis import synthesize, show

In [ ]:
# Hyperparameters
feature_length = 204   # 51 qubits × 4 entries per shadow
num_qubits     = 8
n_layers       = 2
num_weights    = num_qubits * 2 * n_layers   # RX & RY per qubit per layer

@qfunc
def encoding(
    feature: CArray[CReal, feature_length],
    wires:   QArray
) -> None:
    # feature is flat; entries for qubit q are at 4*q … 4*q+3
    for q in range(num_qubits):
        feature_idx = q * (feature_length // num_qubits)
        θ = feature[feature_idx]       # first entry to use
        ϕ = feature[feature_idx + 1]   # second entry to use
        RY(theta=θ, target=wires[q])
        RZ(theta=ϕ, target=wires[q])
@qfunc
def ansatz(
    weights: CArray[CReal, num_weights],
    wires:   QArray
) -> None:
    ptr = 0
    for _ in range(n_layers):
        # per-qubit RX/RY
        for q in range(num_qubits):
            RX(theta=weights[ptr],   target=wires[q]); ptr += 1
            RY(theta=weights[ptr],   target=wires[q]); ptr += 1
        # chain of CZ entanglers
        for q in range(num_qubits - 1):
            CZ(ctrl=wires[q], target=wires[q+1])

@qfunc
def main(
    feature: CArray[CReal, feature_length],
    weights: CArray[CReal, num_weights],
    result:  Output[QArray[QBit]],
) -> None:
    allocate(num_qubits, result)
    encoding(feature, wires=result)
    ansatz(weights, wires=result)

### Synthesis

Before training, you must synthesize your model into a quantum program. Placeholders for your parameters will be automatically generated.

You may find the following documentation useful: https://docs.classiq.io/latest/sdk-reference/synthesis/

In [ ]:
from classiq import create_model, Constraints
from classiq.execution import (
    ExecutionPreferences,
    ClassiqBackendPreferences,
    ClassiqSimulatorBackendNames,
)
from classiq.synthesis import synthesize, show

# Preferences
NUM_SHOTS              = 1000
BACKEND_PREFERENCES    = ClassiqBackendPreferences(
    backend_name=ClassiqSimulatorBackendNames.SIMULATOR
)
OPTIMIZATION_PARAMETER = "no_opt"

# Build & compile
QMOD = create_model(
    main,
    execution_preferences=ExecutionPreferences(
        num_shots=NUM_SHOTS,
        backend_preferences=BACKEND_PREFERENCES
    ),
    constraints=Constraints(optimization_parameter=OPTIMIZATION_PARAMETER)
)
QPROG = synthesize(QMOD)
show(QPROG)


Quantum program link: https://platform.classiq.io/circuit/2xE45bOBlet65QxDevcRo3QcwRG?login=True&version=0.79.1


# Training the Model

Here, you will optimize the weights in ansatz, so that the model can distiguish between the phases.

You can find the following Classiq tutorial and documentation useful:
- Execution: https://docs.classiq.io/latest/sdk-reference/execution/
- Execution Session: https://docs.classiq.io/latest/user-guide/execution/ExecutionSession/
- Executing With Parameters: https://docs.classiq.io/latest/qmod-reference/language-reference/quantum-entry-point/

It is highly recommended to use an ExecutionSession if you are executing the same circuit with different parameters many times. It is not needed to train parameters using the Classiq PyTorch integration.

If you are not using the PyTorch integration, you will need an objective (also known as a 'loss', or 'cost') function. Depending on your implementation, you will need to either minimize or maximize it in training.

In [ ]:
# ────────── Training the Model ────────────────────────────────────────────
import numpy as np
import matplotlib.pyplot as plt
from classiq.execution import ExecutionSession, ExecutionPreferences

def build_session(shots: int, fast: bool) -> ExecutionSession:
    prefs = ExecutionPreferences(
        num_shots=shots,
        backend_preferences=BACKEND_PREFERENCES,
        fast=fast
    )
    return ExecutionSession(quantum_program=QPROG,
                            execution_preferences=prefs)

# 1) Prepare two sessions (32‐shot FAST, then 128‐shot SIMULATOR)
sess32  = build_session(32,  fast=True)
sess128 = build_session(128, fast=False)

# 2) Helper to bind your numpy arrays to the QPROG placeholders
def _bind(prefix: str, arr: np.ndarray):
    # If your qfunc signature is main(feature, weights, …),
    # this will produce { "feature_param_0":…, "weights_param_0":… }
    return { f"{prefix}_param_{i}": float(v) for i, v in enumerate(arr) }

# 3) Convert counts → ⟨Z⟩ on the “last” qubit (or any bit you choose)
def _exp_z0(counts: dict) -> float:
    total = sum(counts.values())
    return sum((+1 if bits[-1]=='0' else -1) * c
               for bits, c in counts.items()) / total

# 4) Batched forward pass returning an array of ⟨Z⟩ for each example
def batch_forward(sess: ExecutionSession,
                  thetas: np.ndarray,   # shape (M, num_weights)
                  Xb:     np.ndarray    # shape (M, feature_dim)
                 ) -> np.ndarray:         # returns shape (M,)
    param_list = [
        { **_bind("weights", th), **_bind("feature", x) }
        for th, x in zip(thetas, Xb)
    ]
    results = sess.batch_sample(parameters=param_list)
    counts  = [
        r.counts if isinstance(r.counts, dict) else r.counts[0]
        for r in results
    ]
    return np.array([_exp_z0(c) for c in counts])

# 5) Hinge‐loss and Accuracy helpers
def hinge_loss(sess, w):
    logits = batch_forward(sess,
                           np.tile(w[None,:], (num_samples,1)),
                           X)
    return np.mean(np.maximum(0, 1 - y * logits))

def accuracy(sess, w):
    logits = batch_forward(sess,
                           np.tile(w[None,:], (num_samples,1)),
                           X)
    return np.mean(np.sign(logits) == y)

# 6) SPSA hyperparameters & initialization
alpha, c0, A, gamma = 0.15, 0.03, 10.0, 0.101
MAX_IT = 100
theta     = 0.01 * np.random.randn(num_weights)
loss_hist = []
acc_hist  = []

# 7) Main SPSA loop with warm‐shot switching
for k in range(MAX_IT):
    sess = sess32 if k < 40 else sess128
    ck   = c0 / (k + 1)**gamma
    ak   = alpha / (k + 1 + A)**0.602

    delta    = np.random.choice([+1, -1], size=num_weights)
    loss_p   = hinge_loss(sess, theta + ck * delta)
    loss_m   = hinge_loss(sess, theta - ck * delta)
    grad_est = (loss_p - loss_m) / (2 * ck * delta)
    theta   -= ak * grad_est

    if k % 10 == 0 or k == MAX_IT - 1:
        L   = hinge_loss(sess, theta)
        Acc = accuracy(sess, theta)
        loss_hist.append(L)
        acc_hist.append(Acc)
        phase = "32-shot" if k < 40 else "128-shot"
        print(f"iter {k:3d} | {phase:<8} | loss={L:.4f} | acc={Acc:.2%}")

# 8) Cleanup
sess32.close()
sess128.close()

# 9) Plot training curves
plt.figure(figsize=(5,2.5))
plt.plot(loss_hist, '-o')
plt.title("Hinge Loss")
plt.xlabel("Checkpoint")
plt.tight_layout()

plt.figure(figsize=(5,2.5))
plt.plot(acc_hist, '-o')
plt.title("Accuracy")
plt.xlabel("Checkpoint")
plt.tight_layout()
plt.show()

# 10) Show final circuit
show(QPROG)


ClassiqAPIError: User provided different parameters than those included in the circuit. Circuit Parameters are {'weights_param_6', 'weights_param_19', 'feature_param_51', 'weights_param_14', 'feature_param_126', 'weights_param_1', 'weights_param_15', 'weights_param_30', 'feature_param_1', 'weights_param_8', 'weights_param_24', 'feature_param_151', 'feature_param_150', 'weights_param_27', 'feature_param_0', 'weights_param_23', 'weights_param_22', 'feature_param_50', 'feature_param_176', 'weights_param_5', 'weights_param_28', 'feature_param_175', 'weights_param_29', 'feature_param_100', 'weights_param_13', 'weights_param_26', 'weights_param_9', 'weights_param_20', 'weights_param_25', 'weights_param_0', 'weights_param_11', 'feature_param_75', 'feature_param_26', 'weights_param_2', 'feature_param_25', 'weights_param_18', 'weights_param_3', 'weights_param_31', 'weights_param_7', 'weights_param_4', 'weights_param_17', 'feature_param_101', 'weights_param_16', 'feature_param_76', 'weights_para

Training that takes too long may make it impossible to grade your submission.

# Testing the Model

Good job! Now it's time to see whether the model you designed can successfully perform the classification. For this, compare the predictions of your model to the actual labels.

If the model does not perform well, try modifying the encoding and/or the ansatz (by using different number of parameters/qubits/ansatz layers/...)

In [ ]:
### Your Code Goes Here: ###

## Grading

You will be evaluated on the accuracy, depth, width of your model and the number of parameters in your model.

The following function will return the width and depth of your model as they will be used in grading. Use it to self-evaluate your model.

In [ ]:
from classiq import QuantumProgram

def get_metrics(qprog):
    """
    Extract circuit metrics from a quantum program.

    Parameters:
        qprog: The quantum program object.

    Returns:
        dict: A dictionary containing the circuit metrics:
              - "depth": Circuit depth
              - "width": Circuit width
    """
    circuit = QuantumProgram.from_qprog(qprog)

    metrics = {
        "depth": circuit.transpiled_circuit.depth,
        "width": circuit.data.width,
    }

    return metrics

In [ ]:
print(get_metrics(QPROG))

# Submission

You will submit this notebook, your trained parameters, and your quantum model.

In [ ]:
# Do not change this cell

import os

def save_qprog(qprog, team_name: str, folder="."):
    assert isinstance(team_name, str)
    file_name = f"{team_name.replace(' ','_')}.qprog"
    with open(os.path.join(folder, file_name), 'w') as f:
        f.write(qprog.model_dump_json(indent=4))

def save_params(params, team_name: str, folder="."):
    assert isinstance(team_name, str)
    file_name = f"{team_name.replace(' ','_')}.npz"
    with open(os.path.join(folder, file_name), 'wb') as f:
        np.savez(f, params=params)

In [ ]:
# Change to your team name!!
TEAM_NAME = ""

# Insert your trained parameters here!
TRAINED_PARAMS = []

save_qprog(QPROG, team_name=TEAM_NAME)
save_params(params=TRAINED_PARAMS, team_name=TEAM_NAME)